In [7]:
import pandas as pd
import numpy as np
import os
import warnings
import random
import plotly.express as px
import plotly.graph_objects as go

In [2]:
warnings.filterwarnings('ignore')
os.chdir('../src/raw_data')

In [3]:

label_df = pd.read_csv('ground_truth.csv')
label_df

,file,recovery,drop
0,00e03657-8e1e-4c8c-a724-1d3c77b48510,"[[2420.9805555555554, 2438.4241666666667], [31...","[[3454.6875, 3764.9605555555554]]"
1,00e4dba2-36d2-42b4-beb1-c55aed75f506,[],"[[13285.465, 19439.800555555557]]"
2,00f035b7-ad7a-4f30-9081-522a3c10805b,[],"[[0.0, 42.75]]"
3,01a0c034-6afc-4e73-95fa-621f702a0b7d,[],"[[0.0, 491.98305555555555]]"
4,01a530d3-6496-4515-9fbb-4f44e298fd29,[],"[[4921.376666666667, 6209.231666666667]]"
...,...,...,...
95,1dfaf03c-e297-4d92-a0bf-40b1a829391f,[],[]
96,1e149fbd-41c6-4779-b87d-c5dc17fbb4c0,[],"[[0.0, 635.3127777777778]]"
97,1e19b77c-8a0e-4749-a384-9c1e679035bf,[],[]
98,1e4b4c18-1e32-45eb-917a-5760e33fbaca,"[[9541.77638888889, 10288.5075]]","[[10339.343055555555, 10739.613055555556], [13..."


In [61]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [64]:
def plot_data(df: pd.DataFrame, indicators_df: pd.DataFrame, filename: str):
    events = indicators_df[indicators_df["file"] == filename][["recovery", "drop"]].values[0]
    recovery_intervals = eval(events[0]) if isinstance(events[0], str) else []
    drop_intervals = eval(events[1]) if isinstance(events[1], str) else []

    fig = px.line(df, x = 'time', y = 'pressure', title = f'Visualization of {filename}')
    fig.update_layout(xaxis_title="Time", yaxis_title="Pressure", template="plotly_white")

    for start, end in recovery_intervals:
        fig.add_vrect(x0=start, x1=end, fillcolor="green", opacity=0.3, layer="below", line_width=0, name="Recovery")
        fig.add_trace(go.Scatter(x=[start, end], y=[df["pressure"].max(), df["pressure"].max()],
                                    mode="lines", line=dict(color="green", width=4), name="Recovery (Bold)"))

    for start, end in drop_intervals:
        fig.add_vrect(x0=start, x1=end, fillcolor="red", opacity=0.3, layer="below", line_width=0, name="Drop")
        fig.add_trace(go.Scatter(x=[start, end], y=[df["pressure"].min(), df["pressure"].min()],
                                    mode="lines", line=dict(color="red", width=4), name="Drop (Bold)"))
    
    fig.update_layout(
        xaxis=dict(rangeslider=dict(visible=True), type="linear"),
        yaxis=dict(fixedrange=False),
        legend=dict(title="Legend", x=0.99, y=0.99, xanchor="right", yanchor="top")
        )
    
    fig.show()

In [75]:
import ast

idx = random.randint(0, 100)
filename = label_df['file'][idx]

if filename in os.listdir('train'):
    file_df = pd.read_csv(f'train/{filename}', sep = '\\s+', names = ['time', 'pressure'])
plot_data(file_df, label_df[label_df['file'] == filename], filename)
print(filename)

0a1c3899-24d5-43aa-9b86-3115b3eeca64


In [ ]:
from scipy.ndimage import uniform_filter1d
file_df['pressure'] =  uniform_filter1d(file_df['pressure'], size = 25)
plot_data(file_df, label_df[label_df['file'] == filename], filename)
print(filename)

0a1c3899-24d5-43aa-9b86-3115b3eeca64


In [60]:
file_df['pressure'] = scaler.fit_transform(file_df[['time', 'pressure']])[:, 1]
plot_data(file_df, label_df[label_df['file'] == filename], filename)
print(filename)

01dd93fd-56c2-4d30-889a-4afd85126784
